In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
from math import log

from tf.app import use
from pocr import PostOcr, showDistribution, showDistributionFreq

Load the corpus

In [3]:
P = PostOcr(4)

Loading TF data ...
Loading tf data from ~/github/annotation/clariah-dr/tf/daghregister/004/0.1
done


Get all the distinct word forms as a dict keyed by the word and valued by the list of slots where it occurs.

In [94]:
F = P.TF.api.F

In [96]:
WORD_OCCS = collections.defaultdict(list)

for w in F.otype.s("word"):
    word = F.letters.v(w)
    WORD_OCCS[word].append(w)
len(WORD_OCCS)

24788

# Components

Make a list of all possible splits of words into two.
Add the unsplit words as well.
Record in how many forms a component occurs

In [26]:
components = collections.Counter()

In [27]:
for word in WORD_OCCS:
    components[word] += 1
    for i in range(1, len(word)):
        comp1 = word[0:i]
        comp2 = word[i:]
        components[comp1] += 1
        components[comp2] += 1

In [28]:
len(components)

140907

In [29]:
max(components.values())

6658

What is the distribution?

We make the distribution dict that for each frequency tells us how many components have that frequency.

In [30]:
compDist = collections.Counter()
for freq in components.values():
    compDist[freq] += 1

In [31]:
showDistributionFreq(compDist, "number of components", "in how many words")

in how many words│number of components
─────────────────┼──────────────────────────────────────────────────────────────────────────────────────
             6658│■1
             5583│■1
             4682│■1
             3926│■1
             3292│■1
             1941│■■■3
             1628│■■2
             1365│■■■■■5
              960│■■■3
              805│■■■■■■■■■9
              675│■■■■4
              566│■■■■■■■7
              475│■■■■■■■7
              398│■■■■■■■■8
              334│■■■■■■■■■■■■■■20
              280│■■■■■■■■■■■12
              235│■■■■■■■■■■■■■■18
              197│■■■■■■■■■■■■■■■■27
              165│■■■■■■■■■■■■■■■■26
              138│■■■■■■■■■■■■■■■■25
              116│■■■■■■■■■■■■■■■■■■32
               97│■■■■■■■■■■■■■■■■■■36
               82│■■■■■■■■■■■■■■■■■■■■47
               68│■■■■■■■■■■■■■■■■■■■■■■63
               57│■■■■■■■■■■■■■■■■■■■■■■59
               48│■■■■■■■■■■■■■■■■■■■■■■61
               40│■■■■■■■■■■■■■■■■■■■■■■■■■■■121
             

We weed oud the components with frequency 1, except those that are complete word forms.

In [32]:
weed = {c for (c, freq) in components.items() if freq == 1 and c not in WORD_OCCS}
len(weed)

90303

In [33]:
components = {x for x in components.items() if x[0] not in weed}
len(components)

50604

We still can produce all words from the components.

We have much more components than words.

We are going to weed out further, in stages.

At all times, we will be able to compose all words from the components we have.